In [1]:
# Load in packages for pandas, astropy, etc. 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.io import ascii
from astropy.table import Column, MaskedColumn
from astropy.io.ascii import masked
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle
from astropy.cosmology import LambdaCDM 
from astropy import cosmology
#from astroquery.simbad import Simbad
#from astroquery.sdss import SDSS

cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7) #Creating our choice of cosmology here...

pd.set_option('display.max_columns', 300) # Setting max number of rows per df to be the size of the df
pd.set_option('display.max_rows', None)

In [2]:
def name_to_coords(df,dfcol):
    if (len(dfcol[0])) == 14:
        df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=6) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=6, stop=13) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=6) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=8) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #print(dfcol.apply(len))
    elif (len(dfcol[0])) == 19:
        df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #elif dfcol.apply(len) ==
    else:
        print('Error Encountered')
        

In [102]:
# reading in the relevant tables here
xliu2011 = ((Table.read('Tables/XLiu2011b/table1.dat', readme = 'Tables/XLiu2011b/ReadMe', format='ascii.cds')).to_pandas())
#print(len(xliu2011))

name_to_coords(xliu2011,xliu2011['SDSS'])

# loading in the N-tuples from the Liu 2011 sample
liu2011triples = pd.read_csv('liu2011_triples.csv', sep=',')
liu2011quad = pd.read_csv('liu2011_quadruples.csv', sep=',')
liu2011quint = pd.read_csv('liu2011_quintuple.csv', sep=',')

# adding triples from Tytler+2009 
tyt2009 = pd.read_csv('Tytler_triples_table_DR1.csv', sep=',')


# loading in the triples from the literature
littriples = pd.read_csv('MAC_individualtriples_singleline.csv', sep=',')


# loading in the quadruples from the literature
litquads = pd.read_csv('MAC_individualquadruples_singleline.csv', sep=',')
#**** Double check the separations

# loading in the quintuples from the literature
litquints = pd.read_csv('MAC_individualquintuples_singleline.csv', sep=',')
#**** check the separations

# loading in the sextuplet from the literature
litsix = pd.read_csv('MAC_individualsextuples_singleline.csv', sep=',')
#**** check separations


In [74]:
# Here we're going to add the relevant columns for the Liu+2011 multiplets
liu2011triples['Primary System Type'] = 'Triple AGN Candidate'
liu2011triples['Secondary System Type'] = 'Dual AGN Candidate'
liu2011triples['Tertiary System Type'] = '-99'
liu2011triples['ST1 Confidence Flag'] = 0.5
liu2011triples['ST2 Confidence Flag'] = 0.5
liu2011triples['ST3 Confidence Flag'] = -99
liu2011triples['Selection Method'] = 'Optical Spectroscopy / Fiber Spectroscopy / Spectroscopic Emission Line Ratios'
liu2011triples['Confirmation Method'] = '-99'

# running through and assigning information for the triples from the Liu 2011 sample
for i in [1,2,3]:
    liu2011triples['z'+str(i)] = -99
    liu2011triples['z'+str(i)+"_type"] = -99
    liu2011triples['Equinox'+str(i)] = "J2000"
    liu2011triples['Coordinate_waveband'+str(i)] = "Optical"
    liu2011triples['Coordinate_Source'+str(i)] = "SDSS"
    liu2011triples['AGN_type'+str(i)] = -99
    for index, row in liu2011.iterrows():
        liu2011triples.loc[liu2011triples['Name'+str(i)]==row['SDSS'], 'z'+str(i)] = row['z']
        liu2011triples.loc[liu2011triples['Name'+str(i)]==row['SDSS'], 'AGN_type'+str(i)] = row['FAGN']

# running through this exercise now for the quadruples
liu2011quad['Primary System Type'] = 'Quadruple AGN Candidate'
liu2011quad['Secondary System Type'] = 'Triple AGN Candidate'
liu2011quad['Tertiary System Type'] = 'Dual AGN Candidate'
liu2011quad['ST1 Confidence Flag'] =0.5
liu2011quad['ST2 Confidence Flag'] = 0.5
liu2011quad['ST3 Confidence Flag'] = 0.5
liu2011quad['Selection Method'] = 'Optical Spectroscopy / Fiber Spectroscopy / Spectroscopic Emission Line Ratios'
liu2011quad['Confirmation Method'] = '-99'

for i in [1,2,3,4]:
    liu2011quad['z'+str(i)] = -99
    liu2011quad['z'+str(i)+"_type"] = "spec"
    liu2011quad['Equinox'+str(i)] = "J2000"
    liu2011quad['Coordinate_waveband'+str(i)] = "Optical"
    liu2011quad['Coordinate_Source'+str(i)] = "SDSS"
    liu2011quad['AGN_type'+str(i)] = -99
    for index, row in liu2011.iterrows():
        liu2011quad.loc[liu2011quad['Name'+str(i)]==row['SDSS'], 'z'+str(i)] = row['z']
        liu2011quad.loc[liu2011quad['Name'+str(i)]==row['SDSS'], 'AGN_type'+str(i)] = row['FAGN']

# aaaaand now for the quintuple from Liu+2011 (note, this object was also the one De Rosa 2015 looked at)
liu2011quint['Primary System Type'] = 'Quintuple AGN Candidate'
liu2011quint['Secondary System Type'] = 'Triple AGN Candidate'
liu2011quint['Tertiary System Type'] = 'Dual AGN Candidate'
liu2011quint['ST1 Confidence Flag'] = 0.5
liu2011quint['ST2 Confidence Flag'] = 0.5
liu2011quint['ST3 Confidence Flag'] = 0.5
liu2011quint['Selection Method'] = 'Optical Spectroscopy / Fiber Spectroscopy / Spectroscopic Emission Line Ratios'
liu2011quint['Confirmation Method'] = '-99'

for i in [1,2,3,4,5]:
    liu2011quint['z'+str(i)] = -99
    liu2011quint['z'+str(i)+"_type"] = "spec"
    liu2011quint['Equinox'+str(i)] = "J2000"
    liu2011quint['Coordinate_waveband'+str(i)] = "Optical"
    liu2011quint['Coordinate_Source'+str(i)] = "SDSS"
    liu2011quint['AGN_type'+str(i)] = -99
    for index, row in liu2011.iterrows():
        liu2011quint.loc[liu2011quint['Name'+str(i)]==row['SDSS'], 'z'+str(i)] = row['z']
        liu2011quint.loc[liu2011quint['Name'+str(i)]==row['SDSS'], 'AGN_type'+str(i)] = row['FAGN']


#xliu_agns = xliu[((xliu['FAGN1']==0) | (xliu['FAGN1']==3) | (xliu['FAGN1']==4) | (xliu['FAGN1']==5)) & \
#                 ((xliu['FAGN2']==0) | (xliu['FAGN2']==3) | (xliu['FAGN2']==4) | (xliu['FAGN2']==5)) & \
#                 (xliu['Sep']>10)]
#
#for index, row in gandalf.iterrows():
#    if row['Name1'] in xliu_agns_1list:
#        #print('True')
#        gandalf.at[index, 'Confidence Flag'] = 1 
#        gandalf.at[index, 'Confirmation Method'] = gandalf.at[index, 'Selection Method']
#        gandalf.at[index, 'Primary System Type'] = 'Dual AGN'
#    elif (row['Paper(s)']=='Liu+2011b') and (row['Name1'] not in xliu_agns_1list):
#        gandalf.at[index, 'Confidence Flag'] = +0.5
#    elif (row['Paper(s)']=='Liu+2011b ; Barrows+2016') and (row['Name1'] not in xliu_agns_1list):
#        gandalf.at[index, 'Confidence Flag'] = +0.5


In [75]:
tyt2009['Primary System Type'] = 'Triple AGN Candidate'
tyt2009['Secondary System Type'] = '-99'
tyt2009['Tertiary System Type'] = '-99'

tyt2009['ST1 Confidence Flag'] = -1
tyt2009['ST2 Confidence Flag'] = -99
tyt2009['ST3 Confidence Flag'] = -99

tyt2009['Confirmation Method'] = '-99'

tyt2009['z1'] = tyt2009['zem1']
tyt2009['z2'] = tyt2009['zem2']
tyt2009['z3'] = tyt2009['zem3']

for index, row in tyt2009.iterrows():
    c1 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011triples.at[index, 'Sep23'] = c1.separation(c2).arcsecond


for index, row in tyt2009.iterrows():
    tyt2009.at[index, 'Sep(kpc)12_z1'] = row['Sep12']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    tyt2009.at[index, 'Sep(kpc)12_z2'] = row['Sep12']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    tyt2009.at[index, 'Sep(kpc)13_z1'] = row['Sep13']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    tyt2009.at[index, 'Sep(kpc)13_z3'] = row['Sep13']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    tyt2009.at[index, 'Sep(kpc)23_z2'] = row['Sep23']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    tyt2009.at[index, 'Sep(kpc)23_z3'] = row['Sep23']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)

#tyt2009
#littriples
#litquads
#litquints
#litsix
#liu2011triples

# and based on the separations alone in the tytler 2009 triples table, we can flag them as 'unlikely' triples

,index,zem1,---1,Pair,m_Pair1,Sep12,RA1,Dec1,zem2,---2,m_Pair2,Sep23,RA2,Dec2,zem3,---3,m_Pair3,Sep13,RA3,Dec3,Equinox1,Coordinate_waveband1,Coordinate_Source1,Equinox2,Coordinate_waveband2,Coordinate_Source2,Equinox3,Coordinate_waveband3,Coordinate_Source3,dV12,dV23,dV13,Paper(s),BibCode(s),DOI(s),Selection Method,RA1_deg,Dec1_deg,RA2_deg,Dec2_deg,RA3_deg,Dec3_deg,Table_flag,Primary System Type,Secondary System Type,Tertiary System Type,ST1 Confidence Flag,ST2 Confidence Flag,ST3 Confidence Flag,Confirmation Method,z1,z2,z3,Sep(kpc)12_z1,Sep(kpc)12_z2,Sep(kpc)13_z1,Sep(kpc)13_z3,Sep(kpc)23_z2,Sep(kpc)23_z3
0,0,2.048,P,8,a,130.92,9:56:58.73,+69:38:52.5,2.054,P,b,108.83,9:57:21.22,+69:37:54.5,2.048,P,c,217.08,9:57:25.94,+69:36:8.5,J2000,NaN,NaN,J2000,NaN,NaN,J2000,NaN,NaN,-588.003365,588.003365,0.000000,Tytler+2009,2009MNRAS.392.1539T,https://doi.org/10.1111/j.1365-2966.2008.14159.x,Optical Spectroscopy,149.244708,69.647917,149.338417,69.631806,149.358083,69.602361,Table2,Triple AGN Candidate,-99,-99,-1,-99,-99,-99,2.048,2.054,2.048,1092.931957,1092.543544,1812.203401,1812.203401,908.199770,908.522647
1,1,2.014,P,56,a,142.19,11:19:22.4,+60:48:51.0,2.293,P,b,164.79,11:19:28.9,+60:46:37.0,2.645,P,c,70.35,11:19:31.1,+60:49:21.0,J2000,NaN,NaN,J2000,NaN,NaN,J2000,NaN,NaN,-26401.799424,-30261.684085,-56161.576186,Tytler+2009,2009MNRAS.392.1539T,https://doi.org/10.1111/j.1365-2966.2008.14159.x,Optical Spectroscopy,169.843333,60.814167,169.870417,60.776944,169.879583,60.822500,Table2,Triple AGN Candidate,-99,-99,-1,-99,-99,-99,2.014,2.293,2.645,1189.336599,1167.324859,588.436808,560.616667,1352.862110,1313.205693
2,2,1.888,P,63,a,387.74,14:58:38.04,+0:24:18.0,1.988,P,b,180.73,14:59:1.28,+0:21:23.7,3.012,P,c,91.46,14:59:7.19,+0:21:1.2,J2000,NaN,NaN,J2000,NaN,NaN,J2000,NaN,NaN,-10174.044490,-85646.073127,-94895.200201,Tytler+2009,2009MNRAS.392.1539T,https://doi.org/10.1111/j.1365-2966.2008.14159.x,Optical Spectroscopy,224.658500,0.405000,224.755333,0.356583,224.779958,0.350333,Table2,Triple AGN Candidate,-99,-99,-1,-99,-99,-99,1.888,1.988,3.012,3263.567751,3247.835788,769.809425,703.707768,1513.852999,1390.565328
3,3,3.073,P,95,a,216.00,8:31:15.9,+38:14:24.0,3.171,P,b,288.08,8:30:53.0,+38:12:43.0,3.149,P,c,417.03,8:30:52.9,+38:9:7.0,J2000,NaN,NaN,J2000,NaN,NaN,J2000,NaN,NaN,-7107.680710,1581.238944,-5527.136604,Tytler+2009,2009MNRAS.392.1539T,https://doi.org/10.1111/j.1365-2966.2008.14159.x,Optical Spectroscopy,127.816250,38.240000,127.720833,38.211944,127.720417,38.151944,Table2,Triple AGN Candidate,-99,-99,-1,-99,-99,-99,3.073,3.171,3.149,1651.876856,1635.681969,3189.269468,3165.023928,2181.515101,2186.365712
4,4,2.490,P,118,a,146.44,16:25:48.0,+26:44:32.5,2.526,P,b,180.33,16:25:48.75,+26:46:58.6,2.605,P,c,129.86,16:25:57.66,+26:44:43.4,J2000,NaN,NaN,J2000,NaN,NaN,J2000,NaN,NaN,-3068.334267,-6624.064320,-9690.195572,Tytler+2009,2009MNRAS.392.1539T,https://doi.org/10.1111/j.1365-2966.2008.14159.x,Optical Spectroscopy,246.450000,26.742361,246.453125,26.782944,246.490250,26.745389,Table2,Triple AGN Candidate,-99,-99,-1,-99,-99,-99,2.490,2.526,2.605,1183.110362,1179.433351,1049.158096,1038.604838,1452.384705,1442.257897


In [103]:
# Now adding in relevant columns for the literature triples table
littriples['Tertiary System Type'] = "-99"
littriples['ST3 Confidence Rank'] = -99

for index, row in littriples.iterrows():
    try:
        c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
        c2 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
        littriples.at[index, 'Sep12'] = c1.separation(c2).arcsecond
        c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
        c2 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
        littriples.at[index, 'Sep13'] = c1.separation(c2).arcsecond
        c1 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
        c2 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
        littriples.at[index, 'Sep23'] = c1.separation(c2).arcsecond
    except: 
        print('Coordinates not available for angular separation calculation.')
        
for index, row in littriples.iterrows():
    try:
        littriples.at[index, 'Sep(kpc)12_z1'] = row['Sep12']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
        littriples.at[index, 'Sep(kpc)12_z2'] = row['Sep12']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
        littriples.at[index, 'Sep(kpc)13_z1'] = row['Sep13']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
        littriples.at[index, 'Sep(kpc)13_z3'] = row['Sep13']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
        littriples.at[index, 'Sep(kpc)23_z2'] = row['Sep23']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
        littriples.at[index, 'Sep(kpc)23_z3'] = row['Sep23']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    except:
        print('Angular separation and/or redshift not available for physical separation calculation.')
        
# we'll need to rewrite the separations for the Roland+2020 target (we're going to overwrite it when calculating \
# separations)

# we need coordinates for Djogovski+ and Schawinski+

littriples



Coordinates not available for angular separation calculation.
Coordinates not available for angular separation calculation.


,Primary System Type,ST1 Confidence Rank,Secondary System Type,ST2 Confidence Rank,Literature Name,Name1,Selection Method,Confirmation Method,z1,z1_type,RA1,Dec1,Equinox1,Coordinate_waveband1,Coordinate_Source1,Brightness1,Brightness_band1,Brightness_type1,Name2,z2,z2_type,RA2,Dec2,Equinox2,Coordinate_waveband2,Coordinate_Source2,Brightness2,Brightness_band2,Brightness_type2,Name3,z3,z3_type,RA3,Dec3,Equinox3,Coordinate_waveband3,Coordinate_Source3,Brightness3,Brightness_band3,Brightness_type3,Sep12,Sep12(kpc),Sep13,Sep13(kpc),Sep23,Sep23(kpc),Paper(s),BibCode(s),DOI(s),Notes,Tertiary System Type,ST3 Confidence Rank,Sep(kpc)12_z1,Sep(kpc)12_z2,Sep(kpc)13_z1,Sep(kpc)13_z3,Sep(kpc)23_z2,Sep(kpc)23_z3
0,Dual/Triple AGN Candidate,-1.0,-99,-99.0,NGC 3341,NGC 3341B Nucleus,Optical Spectroscopy / Optical Spectroscopic E...,NaN,0.027339,spec,10:42:31.47,+05:02:37.63,J2000,X-ray,Chandra,NaN,NaN,NaN,NGC 3341B,0.027154,spec,10:42:32.05,+05:02:41.51,J2000,X-ray,Chandra,NaN,NaN,NaN,NGC 3341C,0.027169,spec,10:42:31.75,+05:02:52.83,J2000,Optical,SDSS,NaN,NaN,NaN,9.495222,5.100,15.765264,6.4,12.175209,8.4,Barth+2008 ; Bianchi+2013,2008ApJ...683L.119B ; 2013MNRAS.435.2335B,https://doi.org/10.1086/591905 ; https://doi.o...,Coordinates are not listed in Barth so they ar...,-99,-99,5.214064,5.179933,8.657101,8.605028,6.641948,6.645497
1,Dual AGN,1.0,Triple AGN Candidate,-0.5,SDSS J1027+1749,J102700.40+174900.8 (SDSS J1027+1749A),Optical Spectroscopic Emission Lines,X-ray Imaging / Optical Long-Slit Spectroscopy,0.066800,spec,10:27:00.40,+17:49:00.8,J2000,Optical,SDSS,NaN,NaN,NaN,J102700.55+174900.2 (SDSS J1027+1749B),0.066400,spec,10:27:00.55,+17:49:00.2,J2000,Optical,SDSS,NaN,NaN,NaN,J102700.38+174902.6 (SDSS J1027+1749C),0.065200,spec,10:27:00.38,+17:49:02.6,J2000,Optical,SDSS,NaN,NaN,NaN,2.224533,3.000,1.822519,NaN,3.413754,2.4,Liu+2011a,2011ApJ...736L...7L,https://doi.org/10.1088/2041-8205/736/1/L7,Triple AGN candidate but Foord+2021 showed the...,-99,-99,2.848160,2.832433,2.333445,2.281830,4.346633,4.274089
2,Triple AGN Candidate,0.5,-99,-99.0,J123652+621354,J123652+621354A,Serendipitous / HST Imaging,NaN,1.355000,spec,12:36:52.77,+62:13:54.8,J2000,NaN,NaN,NaN,NaN,NaN,J123652+621354BC,NaN,NaN,NaN,NaN,J2000,NaN,NaN,NaN,NaN,NaN,J123652+621354D,NaN,NaN,NaN,NaN,J2000,Optical,HST,NaN,NaN,NaN,0.380000,4.500,1.180000,14.0,1.400000,16.9,Schawinski+2011,2011ApJ...743L..37S,10.1088/2041-8205/743/2/L37,No distinct coordinates or redshifts provided ...,-99,-99,3.196277,NaN,9.925282,NaN,NaN,NaN
3,Triple AGN,1.0,-99,-99.0,SDSS J0849+1114,J0849+1114 Gal 1,Mid-Infrared Selection / Optical Spectroscopy,X-ray Imaging / X-ray Spectroscopy / Optical S...,0.770000,spec,08:49:05.529,+11:14:47.876,J2000,0.3-8 keV,Chandra,NaN,NaN,NaN,J0849+1114 Gal 2,0.770000,spec,08:49:05.381,+11:14:47.876,J2000,0.3-8 keV,Chandra,NaN,NaN,NaN,J0849+1114,0.770000,spec,08:49:05.448,+11:14:51.646,J2000,0.3-8 keV,Chandra,NaN,NaN,NaN,2.177369,NaN,3.953855,NaN,3.896730,NaN,Pfeifle+2019b ; Gabányi+2019 ; Liu+2019,2019ApJ...883..167P ; 2019A&A...630L...5G ; ...,https://doi.org/10.3847/1538-4357/ab3a9b ; htt...,NaN,-99,-99,16.131521,16.131521,29.293013,29.293013,28.869789,28.869789
4,Triple AGN Candidate,0.0,Recoil AGN Candidate,0.0,SDSS J1056+5516,SDSS J1056+5516A,Optical Imaging / Optical Spectroscopy,NaN,0.256400,spec,10:56:09.79,+55:16:04.0,J2000,Optical,SDSS,NaN,NaN,NaN,SDSS J1056+5516B,0.257200,spec,10:56:09.48,+55:16:00.9,J2000,Optical,SDSS,NaN,NaN,NaN,SDSS J1056+5516C,0.256600,spec,10:56:09.21,+55:16:04.2,J2000,Optical,SDSS,NaN,NaN,NaN,4.077860,NaN,4.960783,NaN,4.026717,NaN,Kalfountzou+2017,2017ApJ...851L..15K,https://doi.org/10.3847/2041-8213/aa9b2d,NaN,-99,-99,16.243216,16.280215,19.760140,19.771400,16.076038,16.048643
5,Triple AGN,1.0,-99,-99.0,QQQ J1519+0627,QQQ1519A,Optical Spectroscopy / Optical Fiber Spectroscopy,Follow-up Spectroscopy / Near-IR Imaging,1.504000,spec,15:19:47.3,+06:27:53,J2000,Near-IR,Calar Alto,NaN,NaN,NaN,QQQ1519B,1.513000,spec,15:19:45.7,+06:27:52,J2000,Ne

In [97]:
#(10/55)*(93)


In [98]:
#(1/65)*(25)

In [76]:
# now calculating the angular and physical separations for all N-tuples in the Liu+2011 sample
for index, row in liu2011triples.iterrows():
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011triples.at[index, 'Sep12'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011triples.at[index, 'Sep13'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011triples.at[index, 'Sep23'] = c1.separation(c2).arcsecond
    
for index, row in liu2011triples.iterrows():
    liu2011triples.at[index, 'Sep(kpc)12_z1'] = row['Sep12']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    liu2011triples.at[index, 'Sep(kpc)12_z2'] = row['Sep12']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    liu2011triples.at[index, 'Sep(kpc)13_z1'] = row['Sep13']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    liu2011triples.at[index, 'Sep(kpc)13_z3'] = row['Sep13']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    liu2011triples.at[index, 'Sep(kpc)23_z2'] = row['Sep23']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    liu2011triples.at[index, 'Sep(kpc)23_z3'] = row['Sep23']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
 
    
for index, row in liu2011quad.iterrows():
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quad.at[index, 'Sep12'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quad.at[index, 'Sep13'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quad.at[index, 'Sep14'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quad.at[index, 'Sep23'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quad.at[index, 'Sep24'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quad.at[index, 'Sep34'] = c1.separation(c2).arcsecond
    
for index, row in liu2011quad.iterrows():
    liu2011quad.at[index, 'Sep(kpc)12_z1'] = row['Sep12']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quad.at[index, 'Sep(kpc)12_z2'] = row['Sep12']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quad.at[index, 'Sep(kpc)13_z1'] = row['Sep13']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quad.at[index, 'Sep(kpc)13_z3'] = row['Sep13']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quad.at[index, 'Sep(kpc)23_z2'] = row['Sep23']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quad.at[index, 'Sep(kpc)23_z3'] = row['Sep23']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quad.at[index, 'Sep(kpc)14_z1'] = row['Sep14']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quad.at[index, 'Sep(kpc)14_z4'] = row['Sep14']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quad.at[index, 'Sep(kpc)24_z2'] = row['Sep24']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quad.at[index, 'Sep(kpc)24_z3'] = row['Sep24']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quad.at[index, 'Sep(kpc)34_z3'] = row['Sep34']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quad.at[index, 'Sep(kpc)34_z4'] = row['Sep34']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)
 


In [78]:
# and still calculating these for the Liu+2011 sample, but here for the quintuple
for index, row in liu2011quint.iterrows():
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quint.at[index, 'Sep12'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quint.at[index, 'Sep13'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quint.at[index, 'Sep14'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA5'])+' hours').degree,Angle(str(row['Dec5'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quint.at[index, 'Sep15'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quint.at[index, 'Sep23'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quint.at[index, 'Sep24'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA5'])+' hours').degree,Angle(str(row['Dec5'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quint.at[index, 'Sep25'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quint.at[index, 'Sep34'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA5'])+' hours').degree,Angle(str(row['Dec5'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quint.at[index, 'Sep35'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA5'])+' hours').degree,Angle(str(row['Dec5'])+' degrees').degree, unit='deg', frame='icrs')
    liu2011quint.at[index, 'Sep45'] = c1.separation(c2).arcsecond
    

for index, row in liu2011quint.iterrows():
    liu2011quint.at[index, 'Sep(kpc)12_z1'] = row['Sep12']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)12_z2'] = row['Sep12']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)13_z1'] = row['Sep13']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)13_z3'] = row['Sep13']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)14_z1'] = row['Sep14']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)14_z4'] = row['Sep14']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)15_z1'] = row['Sep15']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)15_z5'] = row['Sep15']*cosmo.kpc_proper_per_arcmin(row['z5'])*(u.arcmin/u.kpc)*(1/60)
    #
    liu2011quint.at[index, 'Sep(kpc)23_z2'] = row['Sep23']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)23_z3'] = row['Sep23']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)24_z2'] = row['Sep24']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)24_z4'] = row['Sep24']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)25_z2'] = row['Sep25']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)25_z5'] = row['Sep25']*cosmo.kpc_proper_per_arcmin(row['z5'])*(u.arcmin/u.kpc)*(1/60)
    #
    liu2011quint.at[index, 'Sep(kpc)34_z3'] = row['Sep34']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)34_z4'] = row['Sep34']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)35_z3'] = row['Sep35']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)35_z5'] = row['Sep35']*cosmo.kpc_proper_per_arcmin(row['z5'])*(u.arcmin/u.kpc)*(1/60)
    #
    liu2011quint.at[index, 'Sep(kpc)45_z4'] = row['Sep45']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)
    liu2011quint.at[index, 'Sep(kpc)45_z5'] = row['Sep45']*cosmo.kpc_proper_per_arcmin(row['z5'])*(u.arcmin/u.kpc)*(1/60)



In [64]:
#liu2011quad

,Name1,RA1,Dec1,Name2,RA2,Dec2,Name3,RA3,Dec3,Name4,RA4,Dec4,z1,z1_type,Equinox1,Coordinate_waveband1,Coordinate_Source1,AGN_type1,z2,z2_type,Equinox2,Coordinate_waveband2,Coordinate_Source2,AGN_type2,z3,z3_type,Equinox3,Coordinate_waveband3,Coordinate_Source3,AGN_type3,z4,z4_type,Equinox4,Coordinate_waveband4,Coordinate_Source4,AGN_type4,z5,z5_type,Equinox5,Coordinate_waveband5,Coordinate_Source5,AGN_type5,Sep12,Sep13,Sep14,Sep23,Sep24,Sep34,Sep(kpc)12_z1,Sep(kpc)12_z2,Sep(kpc)13_z1,Sep(kpc)13_z3,Sep(kpc)23_z2,Sep(kpc)23_z3,Sep(kpc)14_z1,Sep(kpc)14_z4,Sep(kpc)24_z2,Sep(kpc)24_z3,Sep(kpc)34_z3,Sep(kpc)34_z4
0,J135632.71+232118.3,13:56:32.71,+23:21:18.3,J135635.73+232136.0,13:56:35.73,+23:21:36.0,J135638.42+232151.5,13:56:38.42,+23:21:51.5,J135639.45+232210.3,13:56:39.45,+23:22:10.3,0.0661,spec,J2000,Optical,SDSS,1,0.0668,spec,J2000,Optical,SDSS,2,0.0659,spec,J2000,Optical,SDSS,2,0.0657,spec,J2000,Optical,SDSS,2,-99,spec,J2000,Optical,SDSS,-99,45.197554,85.351350,106.385871,40.154178,61.647560,23.549803,57.308866,57.868275,108.222428,107.920278,51.411034,50.771898,134.893442,134.140033,78.929889,77.730300,29.776931,29.693523
1,J144702.08+134006.2,14:47:02.08,+13:40:06.2,J144705.78+133953.1,14:47:05.78,+13:39:53.1,J144708.49+133917.0,14:47:08.49,+13:39:17.0,J144712.81+133939.3,14:47:12.81,+13:39:39.3,0.0304,spec,J2000,Optical,SDSS,1,0.0307,spec,J2000,Optical,SDSS,2,0.0285,spec,J2000,Optical,SDSS,2,0.0323,spec,J2000,Optical,SDSS,0,-99,spec,J2000,Optical,SDSS,-99,55.496917,105.592377,158.690871,53.511475,103.391141,66.799826,33.762481,34.083393,64.238894,60.361504,32.864035,30.589643,96.542253,102.342708,63.497597,66.678879,38.185881,43.080456


In [79]:
# Adding relevant columns for the AGN quadruple from Hennawi+2015
litquads['Primary System Type'] = 'Quadruple AGN Candidate'
litquads['Secondary System Type'] = '-99'
litquads['Tertiary System Type'] = '-99'
litquads['ST1 Confidence Flag'] = 1
litquads['ST2 Confidence Flag'] = -99
litquads['ST3 Confidence Flag'] = -99
#litquads['Selection Method'] = 'Optical Spectroscopy / Fiber Spectroscopy / Spectroscopic Emission Line Ratios'
#litquads['Confirmation Method'] = '-99'

# calculating physical and angular separations for the literature quadruples
for index, row in litquads.iterrows():
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    litquads.at[index, 'Sep12'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    litquads.at[index, 'Sep13'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    litquads.at[index, 'Sep14'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    litquads.at[index, 'Sep23'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    litquads.at[index, 'Sep24'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    litquads.at[index, 'Sep34'] = c1.separation(c2).arcsecond

for index, row in litquads.iterrows():
    litquads.at[index, 'Sep(kpc)12_z1'] = row['Sep12']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    litquads.at[index, 'Sep(kpc)12_z2'] = row['Sep12']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    litquads.at[index, 'Sep(kpc)13_z1'] = row['Sep13']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    litquads.at[index, 'Sep(kpc)13_z3'] = row['Sep13']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    litquads.at[index, 'Sep(kpc)23_z2'] = row['Sep23']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    litquads.at[index, 'Sep(kpc)23_z3'] = row['Sep23']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    litquads.at[index, 'Sep(kpc)14_z1'] = row['Sep14']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    litquads.at[index, 'Sep(kpc)14_z4'] = row['Sep14']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)
    litquads.at[index, 'Sep(kpc)24_z2'] = row['Sep24']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    litquads.at[index, 'Sep(kpc)24_z3'] = row['Sep24']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)
    litquads.at[index, 'Sep(kpc)34_z3'] = row['Sep34']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    litquads.at[index, 'Sep(kpc)34_z4'] = row['Sep34']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)




In [80]:
# adding relevant columns for the sextuplet
litsix['Primary System Type'] = 'Sextuplet AGN Candidate'
litsix['Secondary System Type'] = '-99'
litsix['Tertiary System Type'] = '-99'
litsix['ST1 Confidence Flag'] = -1 # here we are flagging it as -1
# NOT because they're not AGNs in a dense environment/protocluster, but because these are NOT the result of \
# mergers and their activity is probably dominated by the large scale environment. 
# Specifically: the physical separations are all >200 kpc between the various AGNs. 
litsix['ST2 Confidence Flag'] = -99
litsix['ST3 Confidence Flag'] = -99
#litsix['Selection Method'] = 'Optical Spectroscopy / Fiber Spectroscopy / Spectroscopic Emission Line Ratios'
#litsix['Confirmation Method'] = '-99'

# calculating physical and angular separations now for the literature sextuplet
for index, row in litsix.iterrows():
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep12'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep13'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep14'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA5'])+' hours').degree,Angle(str(row['Dec5'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep15'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA6'])+' hours').degree,Angle(str(row['Dec6'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep16'] = c1.separation(c2).arcsecond    
    
    c1 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep23'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep24'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA5'])+' hours').degree,Angle(str(row['Dec5'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep25'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA6'])+' hours').degree,Angle(str(row['Dec6'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep26'] = c1.separation(c2).arcsecond
    
    c1 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep34'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA5'])+' hours').degree,Angle(str(row['Dec5'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep35'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA3'])+' hours').degree,Angle(str(row['Dec3'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA6'])+' hours').degree,Angle(str(row['Dec6'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep36'] = c1.separation(c2).arcsecond
    
    c1 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA5'])+' hours').degree,Angle(str(row['Dec5'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep45'] = c1.separation(c2).arcsecond
    c1 = SkyCoord(Angle(str(row['RA4'])+' hours').degree,Angle(str(row['Dec4'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA6'])+' hours').degree,Angle(str(row['Dec6'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep46'] = c1.separation(c2).arcsecond
    
    c1 = SkyCoord(Angle(str(row['RA5'])+' hours').degree,Angle(str(row['Dec5'])+' degrees').degree, unit='deg', frame='icrs')
    c2 = SkyCoord(Angle(str(row['RA6'])+' hours').degree,Angle(str(row['Dec6'])+' degrees').degree, unit='deg', frame='icrs')
    litsix.at[index, 'Sep56'] = c1.separation(c2).arcsecond

    

for index, row in litsix.iterrows():
    litsix.at[index, 'Sep(kpc)12_z1'] = row['Sep12']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)12_z2'] = row['Sep12']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)13_z1'] = row['Sep13']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)13_z3'] = row['Sep13']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)14_z1'] = row['Sep14']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)14_z4'] = row['Sep14']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)15_z1'] = row['Sep15']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)15_z5'] = row['Sep15']*cosmo.kpc_proper_per_arcmin(row['z5'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)16_z1'] = row['Sep16']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)16_z6'] = row['Sep16']*cosmo.kpc_proper_per_arcmin(row['z6'])*(u.arcmin/u.kpc)*(1/60)
    #
    litsix.at[index, 'Sep(kpc)23_z2'] = row['Sep23']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)23_z3'] = row['Sep23']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)24_z2'] = row['Sep24']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)24_z4'] = row['Sep24']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)25_z2'] = row['Sep25']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)25_z5'] = row['Sep25']*cosmo.kpc_proper_per_arcmin(row['z5'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)26_z2'] = row['Sep26']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)26_z6'] = row['Sep26']*cosmo.kpc_proper_per_arcmin(row['z6'])*(u.arcmin/u.kpc)*(1/60)    
    #
    litsix.at[index, 'Sep(kpc)34_z3'] = row['Sep34']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)34_z4'] = row['Sep34']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)35_z3'] = row['Sep35']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)35_z5'] = row['Sep35']*cosmo.kpc_proper_per_arcmin(row['z5'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)36_z3'] = row['Sep36']*cosmo.kpc_proper_per_arcmin(row['z3'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)36_z6'] = row['Sep36']*cosmo.kpc_proper_per_arcmin(row['z6'])*(u.arcmin/u.kpc)*(1/60)
    #
    litsix.at[index, 'Sep(kpc)45_z4'] = row['Sep45']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)45_z5'] = row['Sep45']*cosmo.kpc_proper_per_arcmin(row['z5'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)46_z4'] = row['Sep46']*cosmo.kpc_proper_per_arcmin(row['z4'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)46_z6'] = row['Sep46']*cosmo.kpc_proper_per_arcmin(row['z6'])*(u.arcmin/u.kpc)*(1/60)
    #
    litsix.at[index, 'Sep(kpc)56_z5'] = row['Sep56']*cosmo.kpc_proper_per_arcmin(row['z5'])*(u.arcmin/u.kpc)*(1/60)
    litsix.at[index, 'Sep(kpc)56_z6'] = row['Sep56']*cosmo.kpc_proper_per_arcmin(row['z6'])*(u.arcmin/u.kpc)*(1/60)



In [82]:
#litquads

In [83]:
#liu2011triples

In [84]:
#litsix

In [85]:
#liu2011

In [71]:
## and now for calculating the physical separations.....
#
#for i in [1,2,3,4,5]:
#    liu2011quad['z'+str(i)] = -99
#    liu2011quad['z'+str(i)+"_type"] = "spec"
#    liu2011quad['Equinox'+str(i)] = "J2000"
#    liu2011quad['Coordinate_waveband'+str(i)] = "Optical"
#    liu2011quad['Coordinate_Source'+str(i)] = "SDSS"
#    liu2011quad['AGN_type'+str(i)] = -99
#    for index, row in liu2011.iterrows():
#        liu2011quad.loc[liu2011quad['Name'+str(i)]==row['SDSS'], 'z'+str(i)] = row['z']
#        liu2011quad.loc[liu2011quad['Name'+str(i)]==row['SDSS'], 'AGN_type'+str(i)] = row['FAGN']
#
#
#for index, row in gandalf_white.iterrows():
#    #gandalf_white.at[index, 'dV_new'] = (2.99e+5) * ((1 + row['z1'])**2 - (1 + row['z2'])**2) / ((1 + row['z1'])**2 + (1 + row['z2'])**2)
#    gandalf_white.at[index, 'Sep(kpc)12_z1'] = row['Sep']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
#    gandalf_white.at[index, 'Sep(kpc)12_z2'] = row['Sep']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
#
#
#
#for index, row in gandalf_white.iterrows():
#    #gandalf_white.at[index, 'dV_new'] = (2.99e+5) * ((1 + row['z1'])**2 - (1 + row['z2'])**2) / ((1 + row['z1'])**2 + (1 + row['z2'])**2)
#    gandalf_white.at[index, 'Sep(kpc)_z1'] = row['Sep']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
#    gandalf_white.at[index, 'Sep(kpc)_z2'] = row['Sep']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
#


In [42]:
#xliu_agns_1list = xliu_agns['Name1'].to_list()
#xliu_agns_2list = xliu_agns['Name2'].to_list()
#
#for index, row in gandalf.iterrows():
#    if row['Name1'] in xliu_agns_1list:
#        #print('True')
#        gandalf.at[index, 'Confidence Flag'] = 1 
#        gandalf.at[index, 'Confirmation Method'] = gandalf.at[index, 'Selection Method']
#        gandalf.at[index, 'Primary System Type'] = 'Dual AGN'
#    elif (row['Paper(s)']=='Liu+2011b') and (row['Name1'] not in xliu_agns_1list):
#        gandalf.at[index, 'Confidence Flag'] = +0.5
#    elif (row['Paper(s)']=='Liu+2011b ; Barrows+2016') and (row['Name1'] not in xliu_agns_1list):
#        gandalf.at[index, 'Confidence Flag'] = +0.5



In [ ]:
# # Dropping the quintuple system from the pairs catalog now...
# xliu2011 = xliu2011[xliu2011['SDSS']!='J095859.91+130308.4']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J095900.42+130241.6']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J095903.29+130221.0']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J095906.68+130135.4']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J095908.96+130352.5']

# #Quadruples here
# xliu2011 = xliu2011[xliu2011['SDSS']!='J135632.71+232118.3']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J135635.73+232136.0']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J135638.42+232151.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J135639.45+232210.3']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J144702.08+134006.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J144705.78+133953.1']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J144708.49+133917.0']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J144712.81+133939.3']

# #Dropping all triples
# xliu2011 = xliu2011[xliu2011['SDSS']!='J075925.27+375216.0']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J075931.83+375252.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J075932.30+375319.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J082846.95+122724.1']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J082847.93+122832.0']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J082851.50+122726.8']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J085927.64+173344.9']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J085930.05+173409.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J085931.39+173253.6']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J090515.73+075618.9']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J090516.12+075522.3']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J090520.84+075440.1']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J095942.61+261914.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J095947.21+261711.4']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J095948.15+261845.8']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J100722.96+230234.1']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J100725.21+230114.8']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J100732.36+230131.7']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J102207.11+175017.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J102212.61+175134.8']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J102212.87+175113.6']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J102223.90+154443.1']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J102228.54+154514.0']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J102228.76+154339.0']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J102411.98+025234.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J102415.08+025327.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J102415.56+025241.6']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J102546.26+134300.7']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J102547.75+134456.9']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J102548.60+134340.6']


# xliu2011 = xliu2011[xliu2011['SDSS']!='J105609.41+465341.7']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J105611.76+465250.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J105618.05+465155.4']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J110441.92+041750.4']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J110445.47+041755.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J110449.56+041636.0']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J110517.99+005328.3']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J110520.67+005317.1']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J110521.51+005256.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J110719.72+025820.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J110719.87+025926.0']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J110723.69+025829.3']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J111005.18+032833.7']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J111005.60+032836.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J111007.99+032639.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J114938.17-033408.7']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J114939.73-033146.9']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J114949.79-033103.7']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J120550.07+202837.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J120551.47+203119.1']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J120554.22+203421.1']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J120718.36+204450.9']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J120721.00+204503.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J120722.37+204353.6']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J121425.18+082251.9']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J121428.15+082225.6']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J121431.24+082225.7']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J121442.50+125924.8']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J121442.95+125746.4']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J121443.11+125837.4']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J131628.81+525737.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J131629.82+525601.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J131637.18+525447.6']

# xliu2011 = xliu2011[xliu2011['SDSS']!='J132356.94+305917.0']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J132359.96+305556.7']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J132404.73+305723.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J133227.93+071859.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J133232.26+071739.1']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J133242.55+071938.1']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J141057.24+252950.0']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J141102.54+253109.7']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J141105.14+252857.9']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J141151.61-000303.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J141153.47-000302.3']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J141153.81-000203.0']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J141421.15+025721.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J141422.72+025834.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J141422.99+025840.8']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J143430.61+230529.9']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J143438.51+230443.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J143442.15+230637.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J143801.02+101051.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J143802.18+100944.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J143810.34+100805.8']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J150631.41+404146.4']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J150640.58+404134.9']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J150645.48+404051.5']

# xliu2011 = xliu2011[xliu2011['SDSS']!='J150821.57+191131.6']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J150823.96+191229.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J150831.30+191103.6']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J151310.42+042855.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J151311.70+042701.6']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J151311.80+042952.5']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J151640.00+215938.7']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J151641.90+215955.7']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J151644.79+215932.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J160633.13+174632.3']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J160638.60+174659.8']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J160639.35+174540.3']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J160709.88+183828.6']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J160710.49+184004.7']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J160715.79+184140.8']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J162906.60+114231.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J162907.29+114331.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J162908.94+114447.8']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J163300.85+114510.1']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J163304.69+114431.3']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J163305.33+114532.4']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J231138.88+150741.7']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J231142.06+150638.3']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J231143.99+150603.1']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J231224.52+135704.6']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J231225.99+135450.2']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J231229.22+135632.1']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J232402.98-095341.9']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J232403.39-095334.6']
# xliu2011 = xliu2011[xliu2011['SDSS']!='J232407.99-095334.3']


